In [5]:
from agent import Agent
from referee.game import PlayerColor
agent = Agent(PlayerColor.RED)

agent

Testing: I am playing as red


In [6]:
action = agent.action()
action

SpawnAction(cell=HexPos(r=3, q=3))

In [7]:
from agent.board import MatrixBoard
import numpy as np

matrix = np.zeros([2,7,7], dtype=int)
board = MatrixBoard(matrix, 0, 0, 0)

In [8]:
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir

board.apply_action(SpawnAction(HexPos(1,1)), PlayerColor.RED)
board.apply_action(SpawnAction(HexPos(1,2)), PlayerColor.BLUE)
board.apply_action(SpreadAction(HexPos(1,1), HexDir.UpRight), PlayerColor.RED)

r=%d, q=%d, dr=%d, dq=%d 1 1 1 0


TypeError: MatrixBoard.update_r_q() takes 4 positional arguments but 5 were given

In [ ]:
print(board.render(use_color=True, use_unicode=True))

                         ..     
                     ..      ..     
                 ..      ..      ..     
             ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
     ..      ..      ..      ..      ..      ..     
 ..      r1      ..      ..      ..      ..      ..     
     ..      b1      ..      ..      ..      ..     
         ..      ..      ..      ..      ..     
             ..      ..      ..      ..     
                 ..      ..      ..     
                     ..      ..     
                         ..     

